# 2-1: MLflow Tracking

## 학습 목표
- MLflow Tracking 기본 개념 이해
- Autolog vs 수동 로깅 비교
- Optuna + MLflow 통합 (Nested Runs)
- MLflow UI 사용법

## 이 노트북에서 할 것
```
기존 XGBoost 학습 코드 → MLflow로 실험 추적
                     ↘ 파라미터/메트릭/모델 로깅
                     ↘ Optuna 50 trials 추적
                     ↘ UI에서 실험 비교
```

## 예상 시간: 약 2시간

In [4]:
%%time
# MLflow 설치
!pip install mlflow
!pip install "protobuf>=3.20.0,<5.29.0"

  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl (431 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2
CPU times: total: 31.2 ms
Wall time: 7.03 s


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-sdk 0.77.0 requires protobuf!=5.26.*,!=5.27.*,!=5.28.*,!=5.29.0,!=5.29.1,!=5.29.2,!=5.29.3,!=5.29.4,!=6.30.0,!=6.30.1,!=6.31.0,<7.0,>=4.25.8, but you have protobuf 5.28.3 which is incompatible.


In [39]:
# 패키지 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

# ML 패키지
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, average_precision_score
import joblib
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# MLflow
import mlflow
import mlflow.xgboost
import mlflow.sklearn  # XGBClassifier 로깅용

# 한글 폰트
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 경로
DATA_PROCESSED = Path('../../data/processed')
MODELS_DIR = Path('../../models')

print(f"MLflow 버전: {mlflow.__version__}")
print("패키지 로드 완료!")

MLflow 버전: 3.8.1
패키지 로드 완료!


---

## 데이터 로드 (1-3과 동일)

In [3]:
%%time
# 1-2에서 만든 피처 데이터 로드
train_df = pd.read_csv(DATA_PROCESSED / 'train_features.csv')
test_df = pd.read_csv(DATA_PROCESSED / 'test_features.csv')

print(f"Train: {train_df.shape}")
print(f"Test: {test_df.shape}")

Train: (472432, 448)
Test: (118108, 448)
CPU times: total: 21.8 s
Wall time: 23.8 s


In [5]:
# X, y 분리
X_train = train_df.drop('isFraud', axis=1)
y_train = train_df['isFraud']

X_test = test_df.drop('isFraud', axis=1)
y_test = test_df['isFraud']

# 불균형 비율
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
scale_pos = neg_count / pos_count

print(f"X_train: {X_train.shape}")
print(f"scale_pos_weight: {scale_pos:.1f}")

X_train: (472432, 447)
scale_pos_weight: 27.5


In [6]:
# Validation set 분리 (시간순 20%)
split_idx = int(len(X_train) * 0.8)

X_tr = X_train.iloc[:split_idx]
y_tr = y_train.iloc[:split_idx]
X_val = X_train.iloc[split_idx:]
y_val = y_train.iloc[split_idx:]

print(f"Train: {X_tr.shape}")
print(f"Valid: {X_val.shape}")
print(f"Test: {X_test.shape}")

Train: (377945, 447)
Valid: (94487, 447)
Test: (118108, 447)


### FDS 현업 지표 함수

금융권 FDS에서 추적하는 핵심 지표:
- **Recall (TPR)**: 사기 탐지율 - FDS 핵심!
- **Precision**: 정탐률
- **비용 함수**: FN×손실 + FP×검토비용
- **Recall@FPR**: 특정 FPR에서의 Recall

In [56]:
# FDS 현업 지표 함수 정의
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve
)

def calculate_fds_metrics(y_true, y_prob, threshold=0.5, fn_cost=100, fp_cost=1):
    """
    FDS 현업 표준 지표 계산
    
    Args:
        y_true: 실제 레이블
        y_prob: 예측 확률
        threshold: 분류 임계값 (기본값 0.5, 1-3에서 최적화한 값은 0.18)
        fn_cost: FN 1건당 비용 (사기 놓침 = 큰 손실)
        fp_cost: FP 1건당 비용 (오탐 = 검토 인건비)
    
    Returns:
        dict: FDS 지표들
    """
    y_pred = (y_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    metrics = {
        # Tier 1: 필수 지표
        "auc_roc": roc_auc_score(y_true, y_prob),
        "auprc": average_precision_score(y_true, y_prob),
        "recall": recall_score(y_true, y_pred),      # 사기 탐지율 - FDS 핵심!
        "precision": precision_score(y_true, y_pred, zero_division=0),
        
        # Tier 2: 비용 기반
        "threshold": threshold,
        "cost": int(fn * fn_cost + fp * fp_cost),
        "fn_count": int(fn),
        "fp_count": int(fp),
        
        # Tier 3: 운영 지표
        "f1_score": f1_score(y_true, y_pred, zero_division=0),
        "fpr": fp / (fp + tn) if (fp + tn) > 0 else 0,  # False Positive Rate
        "tp_count": int(tp),
        "tn_count": int(tn),
    }
    
    return metrics


def recall_at_fpr(y_true, y_prob, target_fpr=0.05):
    """
    특정 FPR에서의 Recall 계산
    
    Args:
        y_true: 실제 레이블
        y_prob: 예측 확률
        target_fpr: 목표 FPR (예: 0.05 = 5%)
    
    Returns:
        tuple: (recall, threshold)
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    
    # target_fpr 이하인 지점 찾기
    idx = np.where(fpr <= target_fpr)[0]
    if len(idx) == 0:
        return 0.0, 1.0
    idx = idx[-1]
    return tpr[idx], thresholds[idx]


print("FDS 지표 함수 정의 완료!")
print("- calculate_fds_metrics(): Recall, Precision, 비용 등 계산")
print("- recall_at_fpr(): 특정 FPR에서의 Recall 계산")

FDS 지표 함수 정의 완료!
- calculate_fds_metrics(): Recall, Precision, 비용 등 계산
- recall_at_fpr(): 특정 FPR에서의 Recall 계산


---

## Part 1: MLflow 기초

### 1-1. MLflow란?

**MLflow** = ML 라이프사이클 관리 플랫폼

```
4개 컴포넌트:
1. Tracking: 실험 추적 (파라미터, 메트릭, 아티팩트) ← 이번 노트북
2. Projects: 재현 가능한 패키징
3. Models: 모델 배포 표준화
4. Registry: 모델 버전 관리 ← 2-2에서 다룸
```

### 왜 필요해?

**기존 방식의 문제점:**
```python
# 1-3에서 이렇게 했음...
print(f"AUC: {auc:.4f}")
print(f"best_params: {best_params}")
joblib.dump(model, "model.joblib")

# 문제점:
# - 100개 실험하면 어떤 파라미터가 좋았는지 기억 못함
# - 노트북 재실행하면 결과 날아감
# - 팀원과 결과 공유 어려움
```

**MLflow 사용 시:**
```python
with mlflow.start_run():
    mlflow.log_params(params)     # 파라미터 저장
    mlflow.log_metric("auc", auc) # 메트릭 저장
    mlflow.xgboost.log_model(model, "model")  # 모델 저장

# 장점:
# - 모든 실험 자동 기록
# - UI에서 비교 가능
# - 팀원과 공유 쉬움
```

### 1-2. MLflow 구조

```
실험 (Experiment): "fds-xgboost"
├── Run 1: "xgb_baseline"
│   ├── Parameters: n_estimators=100, max_depth=6
│   ├── Metrics: auc=0.88, auprc=0.45
│   └── Artifacts: model/, feature_importance.png
│
├── Run 2: "xgb_tuned"
│   └── ...
│
└── Run 3: "optuna_50trials" (Parent)
    ├── Child Run 1: trial_0
    ├── Child Run 2: trial_1
    └── ... (Nested Runs)
```

### 1-3. 로컬 설정 (파일 기반)

가장 간단한 방식: 서버 없이 로컬 파일에 저장

In [57]:
# 예제: MLflow 설정 (파일 기반)

# Tracking URI 설정 (로컬 파일)
mlflow.set_tracking_uri("file:./mlruns")

# 실험 생성/선택
experiment = mlflow.set_experiment("fds-xgboost")

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")
print(f"Artifact Location: {experiment.artifact_location}")

Tracking URI: file:./mlruns
Experiment: fds-xgboost
Experiment ID: 537829912260148406
Artifact Location: file:///C:/workspace/fds-system/notebooks/phase2/mlruns/537829912260148406


### [실습 1] MLflow 연결 확인

MLflow를 설정하고 연결을 확인하세요.

In [8]:
# 실습 1: MLflow 연결 확인

# TODO 1: Tracking URI 설정 (파일 기반)
mlflow.set_tracking_uri("_____")

# TODO 2: 실험 생성 (이름: "fds-xgboost-practice")
experiment = mlflow.set_experiment("_____")

# 확인
print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment ID: {experiment.experiment_id}")

2026/01/08 15:14:32 INFO mlflow.tracking.fluent: Experiment with name '_____' does not exist. Creating a new experiment.


Tracking URI: _____
Experiment ID: 129674943858895894


In [58]:
# 실습 1 정답

# 1. Tracking URI 설정
mlflow.set_tracking_uri("file:./mlruns")

# 2. 실험 생성
experiment = mlflow.set_experiment("fds-xgboost-practice")

print(f"Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Experiment: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}")

Tracking URI: file:./mlruns
Experiment: fds-xgboost-practice
Experiment ID: 137883075646855063


In [59]:
# 체크포인트 1: MLflow 연결 확인
assert mlflow.get_tracking_uri() is not None, "Tracking URI 설정 필요!"
assert experiment.experiment_id is not None, "Experiment 생성 실패!"
print("체크포인트 1 통과: MLflow 연결 성공!")

체크포인트 1 통과: MLflow 연결 성공!


---

## Part 2: 기본 Tracking

### 2-1. 수동 로깅 vs Autolog

| 방식 | 장점 | 단점 |
|------|------|------|
| 수동 로깅 | 완전한 제어, 커스텀 메트릭 | 코드량 많음 |
| Autolog | 간편, 자동 로깅 | 제한적 커스터마이징 |

**현업 권장**: Autolog + 필요한 것만 수동 추가

### 2-2. 수동 로깅 예제

In [60]:
# 예제: XGBoost 수동 로깅 (FDS 현업 지표 포함)

# 실험 선택 (다시 메인 실험으로)
mlflow.set_experiment("fds-xgboost")

with mlflow.start_run(run_name="xgb_manual_baseline"):
    # 1. 파라미터 정의 및 로깅
    params = {
        "n_estimators": 100,
        "max_depth": 6,
        "learning_rate": 0.1,
        "scale_pos_weight": scale_pos
    }
    mlflow.log_params(params)
    
    # 2. 모델 학습
    model = XGBClassifier(
        **params,
        random_state=42,
        verbosity=0
    )
    model.fit(X_tr, y_tr)
    
    # 3. 예측
    y_prob_val = model.predict_proba(X_val)[:, 1]
    y_prob_test = model.predict_proba(X_test)[:, 1]
    
    # 4. FDS 현업 지표 계산 (threshold=0.18: 1-3에서 최적화한 값)
    threshold = 0.18
    test_metrics = calculate_fds_metrics(y_test, y_prob_test, threshold=threshold)
    val_metrics = calculate_fds_metrics(y_val, y_prob_val, threshold=threshold)
    
    # Recall@FPR 계산
    recall_5fpr, _ = recall_at_fpr(y_test, y_prob_test, 0.05)
    recall_1fpr, _ = recall_at_fpr(y_test, y_prob_test, 0.01)
    
    # 5. 메트릭 로깅 (FDS 현업 지표)
    mlflow.log_metrics({
        # Validation
        "val_auc": val_metrics["auc_roc"],
        "val_auprc": val_metrics["auprc"],
        "val_recall": val_metrics["recall"],
        
        # Test - Tier 1
        "test_auc": test_metrics["auc_roc"],
        "test_auprc": test_metrics["auprc"],
        "test_recall": test_metrics["recall"],      # FDS 핵심!
        "test_precision": test_metrics["precision"],
        
        # Test - Tier 2 (비용 기반)
        "threshold": threshold,
        "cost": test_metrics["cost"],
        "fn_count": test_metrics["fn_count"],
        "fp_count": test_metrics["fp_count"],
        "recall_at_5pct_fpr": recall_5fpr,
        "recall_at_1pct_fpr": recall_1fpr,
        
        # Test - Tier 3
        "test_f1": test_metrics["f1_score"],
        "n_features": X_train.shape[1]
    })
    
    # 6. 모델 아티팩트 로깅 (joblib 방식)
    import tempfile
    import os
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_path = os.path.join(tmp_dir, "model.joblib")
        joblib.dump(model, model_path)
        mlflow.log_artifact(model_path, artifact_path="model")
    
    run_id_manual = mlflow.active_run().info.run_id
    
    print(f"Run ID: {run_id_manual}")
    print(f"\n=== FDS 핵심 지표 (threshold={threshold}) ===")
    print(f"Test AUC: {test_metrics['auc_roc']:.4f}")
    print(f"Test AUPRC: {test_metrics['auprc']:.4f}")
    print(f"Test Recall: {test_metrics['recall']:.4f} (사기 탐지율)")
    print(f"Test Precision: {test_metrics['precision']:.4f}")
    print(f"비용: {test_metrics['cost']:,} (FN×100 + FP×1)")
    print(f"Recall@5%FPR: {recall_5fpr:.4f}")

Run ID: cd1c9ca6bc41490d8d250ef2b2550ed4

=== FDS 핵심 지표 (threshold=0.18) ===
Test AUC: 0.8872
Test AUPRC: 0.4751
Test Recall: 0.9274 (사기 탐지율)
Test Precision: 0.0683
비용: 80,902 (FN×100 + FP×1)
Recall@5%FPR: 0.5738


### [실습 2] XGBoost 수동 로깅

다른 파라미터로 XGBoost를 학습하고 MLflow에 로깅하세요.

In [40]:
# 실습 2: XGBoost 수동 로깅

# TODO: run 시작 (run_name="xgb_manual_v2")
with mlflow.start_run(run_name="xgb_manual_v2"):
    # TODO: 파라미터 정의 (n_estimators=200, max_depth=8)
    params = {
        "n_estimators": 200,
        "max_depth": 8,
        "learning_rate": 0.1,
        "scale_pos_weight": scale_pos
    }
    
    # TODO: 파라미터 로깅
    mlflow.log_params(params)
    
    # 모델 학습
    model = XGBClassifier(**params, random_state=42, verbosity=0)
    model.fit(X_tr, y_tr)
    
    # TODO: 메트릭 계산
    y_prob_test = model.predict_proba(X_test)[:, 1]
    test_auc = roc_auc_score(y_test, y_prob_test)
    
    # TODO: 메트릭 로깅
    mlflow.log_metric("test_auc", test_auc)
    
    # TODO: 모델 로깅 (sklearn.log_model 사용)
    mlflow.sklearn.log_model(model, name="model")
    
    run_id_v2 = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id_v2}")
    print(f"Test AUC: {test_auc:.4f}")

Run ID: 63901b5e3d514ccfbd19d8153348361e
Test AUC: 0.8732


In [61]:
# 실습 2 정답 (FDS 현업 지표 포함)

with mlflow.start_run(run_name="xgb_manual_v2"):
    # 파라미터 정의
    params = {
        "n_estimators": 200,
        "max_depth": 8,
        "learning_rate": 0.1,
        "scale_pos_weight": scale_pos
    }
    mlflow.log_params(params)
    
    # 모델 학습
    model = XGBClassifier(**params, random_state=42, verbosity=0)
    model.fit(X_tr, y_tr)
    
    # FDS 지표 계산
    y_prob_test = model.predict_proba(X_test)[:, 1]
    threshold = 0.18
    metrics = calculate_fds_metrics(y_test, y_prob_test, threshold=threshold)
    recall_5fpr, _ = recall_at_fpr(y_test, y_prob_test, 0.05)
    
    # FDS 현업 지표 로깅
    mlflow.log_metrics({
        "test_auc": metrics["auc_roc"],
        "test_auprc": metrics["auprc"],
        "test_recall": metrics["recall"],
        "test_precision": metrics["precision"],
        "threshold": threshold,
        "cost": metrics["cost"],
        "recall_at_5pct_fpr": recall_5fpr,
    })
    
    # 모델 저장 (joblib)
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_path = os.path.join(tmp_dir, "model.joblib")
        joblib.dump(model, model_path)
        mlflow.log_artifact(model_path, artifact_path="model")
    
    run_id_v2 = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id_v2}")
    print(f"Test AUC: {metrics['auc_roc']:.4f}")
    print(f"Test Recall: {metrics['recall']:.4f}")
    print(f"비용: {metrics['cost']:,}")

Run ID: 0807c425faa246568151ce84241f18d9
Test AUC: 0.8732
Test Recall: 0.8214
비용: 100,372


In [62]:
# 체크포인트 2: 수동 로깅 확인
run = mlflow.get_run(run_id_v2)
assert "n_estimators" in run.data.params, "파라미터 로깅 실패!"
assert "test_auc" in run.data.metrics, "메트릭 로깅 실패!"
assert float(run.data.metrics["test_auc"]) > 0.8, "AUC가 0.8보다 높아야 합니다!"
print(f"체크포인트 2 통과: AUC = {run.data.metrics['test_auc']:.4f}")

체크포인트 2 통과: AUC = 0.8732


### 2-3. Autolog 사용

`mlflow.xgboost.autolog()`를 사용하면 자동으로 로깅됩니다.

In [63]:
# 예제: XGBoost Autolog (FDS 현업 지표 추가)

# Autolog 활성화 (모델 저장은 비활성화 - XGBClassifier sklearn 래퍼 호환성 문제)
mlflow.xgboost.autolog(
    log_input_examples=False,
    log_model_signatures=True,
    log_models=False
)

with mlflow.start_run(run_name="xgb_autolog"):
    model = XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        scale_pos_weight=scale_pos,
        random_state=42,
        verbosity=0
    )
    
    # eval_set 지정하면 학습 곡선도 로깅됨
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
    # FDS 현업 지표 계산
    y_prob_test = model.predict_proba(X_test)[:, 1]
    threshold = 0.18
    metrics = calculate_fds_metrics(y_test, y_prob_test, threshold=threshold)
    recall_5fpr, _ = recall_at_fpr(y_test, y_prob_test, 0.05)
    
    # FDS 현업 지표 수동 로깅 (Autolog가 못하는 것들)
    mlflow.log_metrics({
        "custom_test_auc": metrics["auc_roc"],
        "custom_test_auprc": metrics["auprc"],
        "custom_test_recall": metrics["recall"],
        "custom_test_precision": metrics["precision"],
        "threshold": threshold,
        "cost": metrics["cost"],
        "recall_at_5pct_fpr": recall_5fpr,
    })
    
    # 모델 수동 저장 (joblib)
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_path = os.path.join(tmp_dir, "model.joblib")
        joblib.dump(model, model_path)
        mlflow.log_artifact(model_path, artifact_path="model")
    
    run_id_autolog = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id_autolog}")
    print(f"Test Recall: {metrics['recall']:.4f} (사기 탐지율)")
    print(f"비용: {metrics['cost']:,}")

# Autolog 비활성화
mlflow.xgboost.autolog(disable=True)

Run ID: bc1ecb0266a74a0aa95a46578713ea5e
Test Recall: 0.9274 (사기 탐지율)
비용: 80,902


### [실습 3] Autolog 비교

Autolog로 학습하고 어떤 것들이 자동 로깅되는지 확인하세요.

In [ ]:
# 실습 3: Autolog 사용

# TODO 1: Autolog 활성화 (모델 저장 비활성화)
mlflow.xgboost.autolog(log_models=False)

# TODO 2: run 시작 (run_name="xgb_autolog_practice")
with mlflow.start_run(run_name="_____"):
    model = XGBClassifier(
        n_estimators=150,
        max_depth=7,
        scale_pos_weight=scale_pos,
        random_state=42,
        verbosity=0
    )
    
    # TODO 3: 학습 (eval_set 포함)
    model.fit(X_tr, y_tr, eval_set=[(_____,  _____)], verbose=False)
    
    # TODO 4: 커스텀 메트릭 추가
    y_prob = model.predict_proba(X_test)[:, 1]
    mlflow.log_metric("custom_auprc", average_precision_score(y_test, y_prob))
    
    # TODO 5: 모델 수동 저장
    mlflow.sklearn.log_model(model, name="_____")
    
    run_id_autolog_practice = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id_autolog_practice}")

# Autolog 비활성화
mlflow.xgboost.autolog(disable=True)

In [52]:
# 실습 3 정답

# Autolog 활성화 (모델 저장 비활성화)
mlflow.xgboost.autolog(log_models=False)

with mlflow.start_run(run_name="xgb_autolog_practice"):
    model = XGBClassifier(
        n_estimators=150,
        max_depth=7,
        scale_pos_weight=scale_pos,
        random_state=42,
        verbosity=0
    )
    
    # 학습
    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
    # 커스텀 메트릭 추가
    y_prob = model.predict_proba(X_test)[:, 1]
    mlflow.log_metric("custom_auprc", average_precision_score(y_test, y_prob))
    
    # 모델 수동 저장 (joblib + log_artifact - XGBClassifier sklearn 래퍼 호환성)
    import tempfile
    import os
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_path = os.path.join(tmp_dir, "model.joblib")
        joblib.dump(model, model_path)
        mlflow.log_artifact(model_path, artifact_path="model")
    
    run_id_autolog_practice = mlflow.active_run().info.run_id
    print(f"Run ID: {run_id_autolog_practice}")

# Autolog 비활성화
mlflow.xgboost.autolog(disable=True)

Run ID: 817450f1ff594795a40662b0c1c22da7


In [53]:
# 체크포인트 3: Autolog 아티팩트 확인
from mlflow.tracking import MlflowClient
client = MlflowClient()

artifacts = client.list_artifacts(run_id_autolog_practice)
artifact_names = [a.path for a in artifacts]

print(f"저장된 아티팩트: {artifact_names}")

# model 폴더 확인 (joblib 저장 방식)
assert "model" in artifact_names, "모델 아티팩트 없음! 모델 저장 실패!"

# model 폴더 내용 확인
model_artifacts = client.list_artifacts(run_id_autolog_practice, path="model")
model_files = [a.path for a in model_artifacts]
print(f"model 폴더 내용: {model_files}")
assert any("model.joblib" in f for f in model_files), "model.joblib 파일 없음!"

print("체크포인트 3 통과: 모델 저장 성공!")

저장된 아티팩트: ['feature_importance_weight.json', 'feature_importance_weight.png', 'model']
model 폴더 내용: ['model/model.joblib']
체크포인트 3 통과: 모델 저장 성공!


---

## Part 3: Optuna + MLflow 통합

### 3-1. Nested Runs 개념

```
Nested Runs = 부모-자식 구조

Parent Run: "optuna_tuning_50trials"
├── Child Run 1: trial_0 (n_estimators=150, AUC=0.88)
├── Child Run 2: trial_1 (n_estimators=200, AUC=0.89)
├── ...
└── Child Run 50: trial_49 (n_estimators=180, AUC=0.91)

장점:
- 실험 정리가 깔끔함
- 부모 run에 best_params 요약 가능
- UI에서 계층적 뷰
```

### 3-2. Optuna Callback 패턴

In [ ]:
# 예제: Optuna + MLflow (10 trials - 데모용)

def objective_demo(trial):
    """Optuna objective with MLflow logging"""
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}"):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 300),
            "max_depth": trial.suggest_int("max_depth", 4, 10),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
            "scale_pos_weight": scale_pos,
            "random_state": 42,
            "verbosity": 0
        }
        
        # 파라미터 로깅
        mlflow.log_params(params)
        
        # 모델 학습
        model = XGBClassifier(**params)
        model.fit(X_tr, y_tr)
        
        # 메트릭 계산
        y_prob = model.predict_proba(X_val)[:, 1]
        auprc = average_precision_score(y_val, y_prob)
        auc = roc_auc_score(y_val, y_prob)
        
        # 메트릭 로깅
        mlflow.log_metrics({"auprc": auprc, "auc": auc})
        
        return auprc

# Parent Run
with mlflow.start_run(run_name="optuna_demo_10trials"):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_demo, n_trials=10, show_progress_bar=True)
    
    # Best 결과를 Parent에 로깅
    mlflow.log_params({f"best_{k}": v for k, v in study.best_params.items()})
    mlflow.log_metric("best_auprc", study.best_value)
    
    parent_run_id_demo = mlflow.active_run().info.run_id
    print(f"\nParent Run ID: {parent_run_id_demo}")
    print(f"Best AUPRC: {study.best_value:.4f}")
    print(f"Best params: {study.best_params}")

### [실습 4] Optuna + MLflow 50 trials (GPU)

1-3과 동일한 설정으로 Optuna 튜닝을 수행하고 MLflow로 추적합니다.

In [ ]:
%%time
# 실습 4: Optuna + MLflow 50 trials (GPU)

def objective(trial):
    """Optuna objective with MLflow logging (GPU)"""
    # TODO 1: nested run 시작
    with mlflow.start_run(nested=_____, run_name=f"trial_{trial.number}"):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 4, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "scale_pos_weight": scale_pos,
            "tree_method": "hist",
            "device": "cuda",  # GPU 사용
            "random_state": 42,
            "verbosity": 0
        }
        
        # TODO 2: 파라미터 로깅
        mlflow._____
        
        # 모델 학습
        model = XGBClassifier(**params)
        model.fit(X_tr, y_tr)
        
        # TODO 3: AUPRC 계산
        y_prob = model.predict_proba(X_val)[:, 1]
        auprc = _____
        auc = roc_auc_score(y_val, y_prob)
        
        # TODO 4: 메트릭 로깅
        mlflow.log_metrics({"auprc": _____, "auc": _____})
        
        return auprc

# TODO 5: Parent Run 시작
with mlflow.start_run(run_name="_____"):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, show_progress_bar=True)
    
    # TODO 6: best params 로깅
    mlflow.log_params({f"best_{k}": v for k, v in _____.items()})
    mlflow.log_metric("best_auprc", _____)
    
    parent_run_id = mlflow.active_run().info.run_id
    
print(f"\nParent Run ID: {parent_run_id}")
print(f"Best AUPRC: {study.best_value:.4f}")

In [ ]:
%%time
# 실습 4 정답 (FDS 현업 지표 포함)

def objective(trial):
    """Optuna objective with MLflow logging (GPU) + FDS 지표"""
    with mlflow.start_run(nested=True, run_name=f"trial_{trial.number}"):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 500),
            "max_depth": trial.suggest_int("max_depth", 4, 12),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "scale_pos_weight": scale_pos,
            "tree_method": "hist",
            "device": "cuda",
            "random_state": 42,
            "verbosity": 0
        }
        
        mlflow.log_params(params)
        
        model = XGBClassifier(**params)
        model.fit(X_tr, y_tr)
        
        # Validation FDS 지표 계산
        y_prob = model.predict_proba(X_val)[:, 1]
        threshold = 0.18
        metrics = calculate_fds_metrics(y_val, y_prob, threshold=threshold)
        recall_5fpr, _ = recall_at_fpr(y_val, y_prob, 0.05)
        
        # FDS 현업 지표 로깅 (Validation)
        mlflow.log_metrics({
            "auc": metrics["auc_roc"],
            "auprc": metrics["auprc"],
            "recall": metrics["recall"],
            "precision": metrics["precision"],
            "cost": metrics["cost"],
            "recall_at_5pct_fpr": recall_5fpr,
        })
        
        return metrics["auprc"]  # AUPRC 최적화

with mlflow.start_run(run_name="optuna_gpu_50trials"):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50, show_progress_bar=True)
    
    # Best params를 Parent에 로깅
    mlflow.log_params({f"best_{k}": v for k, v in study.best_params.items()})
    mlflow.log_metric("best_auprc", study.best_value)
    
    parent_run_id = mlflow.active_run().info.run_id
    
print(f"\nParent Run ID: {parent_run_id}")
print(f"Best AUPRC: {study.best_value:.4f}")
print(f"Best params: {study.best_params}")

In [45]:
# 체크포인트 4: Nested runs 확인
client = MlflowClient()

# fds-xgboost 실험에서 검색 (Optuna가 실행된 실험)
fds_experiment = mlflow.get_experiment_by_name("fds-xgboost")

# parent run의 child runs 검색
child_runs = client.search_runs(
    experiment_ids=[fds_experiment.experiment_id],
    filter_string=f"tags.mlflow.parentRunId = '{parent_run_id}'"
)

print(f"Child runs 개수: {len(child_runs)}")
assert len(child_runs) == 50, f"50개 trial 필요, 현재 {len(child_runs)}개!"
print(f"체크포인트 4 통과: {len(child_runs)}개 nested runs 생성!")

Child runs 개수: 50
체크포인트 4 통과: 50개 nested runs 생성!


### 3-3. Best Model 저장

In [ ]:
# Best params로 최종 모델 학습 및 저장 (FDS 현업 지표 포함)

best_params = study.best_params.copy()
best_params.update({
    "scale_pos_weight": scale_pos,
    "tree_method": "hist",
    "device": "cuda",
    "random_state": 42,
    "verbosity": 0
})

with mlflow.start_run(run_name="best_model_final"):
    # 전체 Train으로 학습
    final_model = XGBClassifier(**best_params)
    final_model.fit(X_train, y_train)
    
    # Test FDS 지표 계산
    y_prob_test = final_model.predict_proba(X_test)[:, 1]
    threshold = 0.18  # 1-3에서 최적화한 값
    metrics = calculate_fds_metrics(y_test, y_prob_test, threshold=threshold)
    recall_5fpr, _ = recall_at_fpr(y_test, y_prob_test, 0.05)
    recall_1fpr, _ = recall_at_fpr(y_test, y_prob_test, 0.01)
    
    # 파라미터 로깅
    mlflow.log_params(best_params)
    
    # FDS 현업 지표 로깅 (전체)
    mlflow.log_metrics({
        # Tier 1: 필수 지표
        "test_auc": metrics["auc_roc"],
        "test_auprc": metrics["auprc"],
        "test_recall": metrics["recall"],          # FDS 핵심!
        "test_precision": metrics["precision"],
        
        # Tier 2: 비용 기반
        "threshold": threshold,
        "cost": metrics["cost"],
        "fn_count": metrics["fn_count"],
        "fp_count": metrics["fp_count"],
        "recall_at_5pct_fpr": recall_5fpr,
        "recall_at_1pct_fpr": recall_1fpr,
        
        # Tier 3: 운영 지표
        "test_f1": metrics["f1_score"],
        "fpr": metrics["fpr"],
    })
    
    # 모델 저장 (joblib)
    with tempfile.TemporaryDirectory() as tmp_dir:
        model_path = os.path.join(tmp_dir, "model.joblib")
        joblib.dump(final_model, model_path)
        mlflow.log_artifact(model_path, artifact_path="best_model")
    
    # Feature Importance 시각화
    fig, ax = plt.subplots(figsize=(10, 8))
    importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': final_model.feature_importances_
    }).sort_values('importance', ascending=False).head(20)
    
    ax.barh(importance['feature'], importance['importance'])
    ax.set_xlabel('Importance')
    ax.set_title('Top 20 Feature Importance')
    ax.invert_yaxis()
    plt.tight_layout()
    
    mlflow.log_figure(fig, "feature_importance.png")
    plt.close()
    
    best_run_id = mlflow.active_run().info.run_id
    
print(f"Best Model Run ID: {best_run_id}")
print(f"\n=== FDS 핵심 지표 (threshold={threshold}) ===")
print(f"Test AUC: {metrics['auc_roc']:.4f}")
print(f"Test AUPRC: {metrics['auprc']:.4f}")
print(f"Test Recall: {metrics['recall']:.4f} (사기 탐지율)")
print(f"Test Precision: {metrics['precision']:.4f}")
print(f"비용: {metrics['cost']:,} (FN×100 + FP×1)")
print(f"Recall@5%FPR: {recall_5fpr:.4f}")
print(f"Recall@1%FPR: {recall_1fpr:.4f}")

### [실습 5] 최적 모델 아티팩트 저장

In [47]:
# 체크포인트 5: Best 파라미터 로깅 확인
best_run = mlflow.get_run(best_run_id)

assert "n_estimators" in best_run.data.params, "Best params 로깅 실패!"
assert "test_auc" in best_run.data.metrics, "Test metric 로깅 실패!"
assert float(best_run.data.metrics["test_auc"]) > 0.85, "AUC가 0.85보다 높아야 합니다!"

print(f"체크포인트 5 통과!")
print(f"  Test AUC: {best_run.data.metrics['test_auc']:.4f}")
print(f"  Test AUPRC: {best_run.data.metrics['test_auprc']:.4f}")

체크포인트 5 통과!
  Test AUC: 0.9026
  Test AUPRC: 0.5434


---

## Part 4: MLflow UI 활용

### 4-1. UI 실행 방법

터미널에서 실행:
```bash
cd notebooks/phase2
mlflow ui --port 5000
```

브라우저에서 http://localhost:5000 접속

### 4-2. UI 기능

1. **실험 목록**: 왼쪽 사이드바에서 실험 선택
2. **Run 비교**: 체크박스로 여러 run 선택 → "Compare" 버튼
3. **메트릭 시각화**: Run 상세 → Metrics 탭
4. **아티팩트 확인**: Run 상세 → Artifacts 탭
5. **Nested Runs**: Parent run 클릭 → Child runs 목록

In [48]:
# MLflow UI 실행 안내
print("="*60)
print("MLflow UI 실행 방법")
print("="*60)
print("\n1. 새 터미널 열기")
print("\n2. 다음 명령어 실행:")
print("   cd notebooks/phase2")
print("   mlflow ui --port 5000")
print("\n3. 브라우저에서 접속:")
print("   http://localhost:5000")
print("\n4. 확인할 것:")
print("   - fds-xgboost 실험 선택")
print("   - optuna_gpu_50trials run 클릭")
print("   - 50개 nested runs 확인")
print("   - best_model_final의 아티팩트 확인")

MLflow UI 실행 방법

1. 새 터미널 열기

2. 다음 명령어 실행:
   cd notebooks/phase2
   mlflow ui --port 5000

3. 브라우저에서 접속:
   http://localhost:5000

4. 확인할 것:
   - fds-xgboost 실험 선택
   - optuna_gpu_50trials run 클릭
   - 50개 nested runs 확인
   - best_model_final의 아티팩트 확인


### [실습 6] UI 탐색

MLflow UI를 열고 다음을 확인하세요:

1. [ ] `fds-xgboost` 실험에 여러 run이 있는지 확인
2. [ ] `optuna_gpu_50trials` run의 nested runs (50개) 확인
3. [ ] `best_model_final`의 아티팩트 (`best_model/`, `feature_importance.png`) 확인
4. [ ] 여러 run을 선택하고 Compare로 메트릭 비교

In [54]:
# 실험 요약 출력
print("="*60)
print("실험 요약")
print("="*60)

# fds-xgboost 실험에서 조회
fds_experiment = mlflow.get_experiment_by_name("fds-xgboost")

# 모든 runs 조회 후 Python에서 Parent runs만 필터링
# (MLflow filter에서 "태그가 없음"을 표현하기 어려움)
all_runs = client.search_runs(
    experiment_ids=[fds_experiment.experiment_id],
    order_by=["start_time DESC"]
)

# Parent runs = mlflow.parentRunId 태그가 없는 runs
parent_runs = [
    run for run in all_runs 
    if "mlflow.parentRunId" not in run.data.tags
]

print(f"\n총 {len(parent_runs)}개 Parent Runs:")
for run in parent_runs[:10]:  # 최근 10개만
    run_name = run.data.tags.get("mlflow.runName", "unnamed")
    test_auc = run.data.metrics.get("test_auc", run.data.metrics.get("custom_test_auc", "N/A"))
    if isinstance(test_auc, float):
        print(f"  - {run_name}: AUC={test_auc:.4f}")
    else:
        print(f"  - {run_name}: AUC={test_auc}")

실험 요약

총 24개 Parent Runs:
  - xgb_autolog_practice: AUC=N/A
  - xgb_manual_v2: AUC=0.8732
  - best_model_final: AUC=0.9026
  - optuna_demo_10trials: AUC=N/A
  - xgb_autolog_practice: AUC=N/A
  - xgb_manual_v2: AUC=0.8732
  - xgb_autolog_practice: AUC=N/A
  - optuna_gpu_50trials: AUC=N/A
  - optuna_demo_10trials: AUC=N/A
  - xgb_autolog_practice: AUC=N/A


<cell_type>markdown</cell_type>---

## 결론 및 면접 Q&A

### Q: "MLflow를 왜 도입했나요?"

> "모델 학습 시 파라미터, 메트릭, 모델 파일을 체계적으로 관리하기 위해 도입했습니다.
> 기존에는 노트북에서 결과를 수동으로 기록했는데, 실험이 많아지면서 비교가 어려웠습니다.
> MLflow Tracking으로 모든 실험을 자동 기록하고, UI에서 바로 비교할 수 있게 됐습니다."

### Q: "FDS에서 어떤 지표를 추적하나요?" ⭐

> "AUC-ROC와 AUPRC 외에도 **Recall(사기 탐지율)**을 핵심으로 추적합니다.
> FDS에서는 사기를 놓치면(FN) 금전적 손실이 크기 때문에 Recall이 가장 중요합니다.
> 추가로 **비용 함수**(FN×100 + FP×1)로 실제 비즈니스 임팩트를 측정하고,
> **Recall@5%FPR**로 오탐을 제한하면서 얼마나 많은 사기를 잡는지 확인합니다.
> threshold=0.18은 비용 함수를 최소화하는 값으로, 기본값 0.5 대비 비용을 48% 절감합니다."

### Q: "Autolog vs 수동 로깅, 언제 뭘 쓰나요?"

> "기본적으로 Autolog를 사용하고, 필요한 커스텀 메트릭만 수동 추가합니다.
> 예를 들어 XGBoost Autolog는 기본 메트릭을 자동 로깅하지만,
> FDS에서 중요한 Recall, 비용 함수, Recall@FPR은 수동으로 추가했습니다."

### Q: "Optuna와 MLflow를 어떻게 통합했나요?"

> "Nested Runs 패턴을 사용했습니다.
> Parent Run에서 Optuna study를 실행하고, 각 trial마다 Child Run을 생성합니다.
> 50개 trial이 끝나면 Parent Run에 best_params와 best_score를 요약 저장합니다.
> 각 trial에서도 Recall, 비용 등 FDS 지표를 로깅해서 비교할 수 있습니다."

### Q: "프로덕션에서는 어떻게 사용하나요?"

> "로컬에서는 파일 기반(`file:./mlruns`)을 사용했지만,
> 프로덕션에서는 PostgreSQL + S3를 backend로 사용합니다.
> Model Registry로 모델 버전을 관리하고, CI/CD 파이프라인에서 자동 배포합니다.
> 이건 다음 노트북(2-2)에서 다룹니다."

---

## 로깅된 FDS 지표 요약

| 지표 | 설명 | 중요도 |
|------|------|--------|
| test_recall | 사기 탐지율 (FDS 핵심!) | ⭐⭐⭐ |
| test_precision | 정탐률 | ⭐⭐ |
| test_auprc | 불균형 데이터 핵심 지표 | ⭐⭐⭐ |
| test_auc | 전체 성능 | ⭐⭐ |
| cost | FN×100 + FP×1 | ⭐⭐⭐ |
| recall_at_5pct_fpr | FPR 5%에서 Recall | ⭐⭐ |
| threshold | 분류 임계값 | ⭐⭐ |

---

## 다음 단계

**2-2: MLflow Model Registry**
- 모델 버전 관리
- Staging → Production 워크플로
- 모델 로드 및 서빙

In [55]:
# 최종 체크리스트
print("="*60)
print("2-1 MLflow Tracking 체크리스트")
print("="*60)

checks = [
    ("MLflow 연결 완료", mlflow.get_tracking_uri() is not None),
    ("수동 로깅 완료", 'run_id_v2' in dir()),
    ("Autolog 완료", 'run_id_autolog_practice' in dir()),
    ("Optuna 50 trials 완료", 'parent_run_id' in dir()),
    ("Best model 저장 완료", 'best_run_id' in dir()),
]

all_passed = True
for name, passed in checks:
    status = "OK" if passed else "X"
    print(f"  [{status}] {name}")
    if not passed:
        all_passed = False

print("="*60)
if all_passed:
    print("모든 체크 통과! 2-1 완료!")
else:
    print("일부 체크 실패 - 위 내용 확인 필요")

2-1 MLflow Tracking 체크리스트
  [OK] MLflow 연결 완료
  [OK] 수동 로깅 완료
  [OK] Autolog 완료
  [OK] Optuna 50 trials 완료
  [OK] Best model 저장 완료
모든 체크 통과! 2-1 완료!
